In [350]:
import pandas as pd
import numpy as np

In [351]:
data = pd.read_csv('./train.csv')

# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'



In [352]:
print(data.shape)
data.head(10)

(5282, 20)


,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0
5,25,19.80,475.2,Female,0,No,No,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
6,27,90.15,2423.4,Female,0,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,Yes,Month-to-month,No,Bank transfer (automatic),0
7,1,45.70,45.7,Male,0,No,No,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,1
8,50,105.95,5341.8,Male,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,No,Yes,No,Yes,Yes,Month-to-month,No,Credit card (automatic),1
9,72,61.20,4390.25,Male,0,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),0


In [353]:
#удаление записей с пробелами
empty_indexes = data.loc[data['TotalSpent'] == ' '].index
data = data.drop(empty_indexes)
data["TotalSpent"] = data["TotalSpent"].astype(np.float64)

In [354]:
# кодирование категориальных признаков и разбиение на обучающую и валидационную выборки
y_data = data[target_col]
X_data = data.drop(target_col, axis=1)

X_cat = X_data[cat_cols]
X_num = X_data[num_cols]

X_cat = pd.get_dummies(X_cat)
X_cat = np.array(X_cat) 

X = np.hstack((X_num,X_cat))
y = np.array(y_data)

X_train = []
X_valid = []
y_train = []
y_valid = []

for i in range(1,len(X)):
    if i % 5 == 0:
        X_valid.append(X[i])
        y_valid.append(y[i])
    else:
        X_train.append(X[i])
        y_train.append(y[i])

X_train = np.array(X_train)
X_valid = np.array(X_valid)
y_train = np.array(y_train)
y_valid = np.array(y_valid)

print(X_train.shape)

(4218, 45)


In [355]:
#классы слоев модели

# полносвязный слой
class Dense:
    def __init__(self,input_size,output_size):
        self.input_size = input_size
        self.output_size = output_size
        
        self.dW = None
        self.db = None
        self.dX = None
        self.w = np.random.rand(input_size, output_size) - 0.5
        self.b = np.random.rand(output_size) - 0.5
        
    def __call__(self,X):
        if X.shape[1] != self.input_size and X.shape[0] == X.input_size:
            X = X.T
            
        res = X@self.w + self.b
        self.h = res
        return res
    
# активационная функция слоя (Tanh)
class Tanh:
    def __call__(self, X):
        self.dt = None
        self.h = (np.exp(X) - np.exp(-X))/(np.exp(X) + np.exp(-X))
        return self.h
    
# Sigmoid
class Sigm:
    def __call__(self, X):
        self.dt = None
        self.h = 1/(1 + np.exp(-X))
        return self.h
    
# loss функция для данной задачи - бинарная кросс-энтропия
class Loss:
    def __init__(self):
        self.dL = None
    
    def __call__(self,X,y):
        loss_value = 0
        for i in range(len(X)):
            loss_value -= y[i]*np.log(X[i]) + (1-y[i])*np.log(1-X[i]) 
        return loss_value

# вычисление сигмоиды
def sigm(x):
    return 1/(1+np.exp(-x))

In [356]:
#модель
class Model:
    def __init__(self, input_size, lr):
        self.fc1 = Dense(input_size,256)
        self.act1 = Sigm()
        
        self.fc2 = Dense(256,512)
        self.act2 = Sigm()
        
        self.fc3 = Dense(512,128)
        self.act3 = Sigm()
        
        self.fc4 = Dense(128,1)
        self.act4 = Sigm()
        
        self.lr = lr
        self.loss = Loss()
        
        
    # прямой проход по моделе(предсказание). Запоминаются выходы на каждом слое
    def forward(self,X):
        self.X = X
        res = self.fc1(X)
        res = self.act1(res)
        res = self.fc2(res)
        res = self.act2(res)
        res = self.fc3(res)
        res = self.act3(res)
        res = self.fc4(res)
        res = self.act4(res)
        return res
    
    #обратное рспространение ошибки - вычисление градиентов для кажого слоя
    def backward(self, target):
        self.loss.dL = -target/self.act4.h.T + (1-target)/(1-self.act4.h.T)
        self.act4.dt = self.loss.dL.T * (sigm(self.fc4.h)*(1-sigm(self.fc4.h)))

        self.fc4.db = np.sum(self.act4.dt,axis=0,keepdims=True)
        self.fc4.dW = self.act3.h.T @ self.act4.dt
        self.fc4.dX = self.act4.dt @ self.fc4.w.T

        self.act3.dt = self.fc4.dX * (sigm(self.fc3.h)*(1-sigm(self.fc3.h)))

        self.fc3.db = np.sum(self.act3.dt,axis=0,keepdims=True)
        self.fc3.dW = self.act2.h.T @ self.act3.dt
        self.fc3.dX = self.act3.dt @ self.fc3.w.T

        self.act2.dt = self.fc3.dX * (sigm(self.fc2.h)*(1-sigm(self.fc2.h)))

        self.fc2.db = np.sum(self.act2.dt,axis=0,keepdims=True)
        self.fc2.dW = self.act1.h.T @ self.act2.dt
        self.fc2.dX = self.act2.dt @ self.fc2.w.T

        self.act1.dt = self.fc2.dX *  (sigm(self.fc1.h)*(1-sigm(self.fc1.h)))

        self.fc1.db = np.sum(self.act1.dt,axis=0,keepdims=True)
        self.fc1.dW = self.X.T @ self.act1.dt
        
        self.X = None
       
    #шаг градиентного спуска, обнуление градиентов
    def step(self):
        self.fc1.w -= self.lr*self.fc1.dW
        self.fc1.b -= self.lr*self.fc1.db[0]
        self.fc2.w -= self.lr*self.fc2.dW
        self.fc2.b -= self.lr*self.fc2.db[0]
        self.fc3.w -= self.lr*self.fc3.dW
        self.fc3.b -= self.lr*self.fc3.db[0]
        self.fc4.w -= self.lr*self.fc4.dW
        self.fc4.b -= self.lr*self.fc4.db[0]

        self.fc1.dW = None
        self.fc1.db = None
        self.fc1.dX = None
        self.fc2.dW = None
        self.fc2.db = None
        self.fc2.dX = None
        self.fc3.dX = None
        self.fc3.db = None
        self.fc3.dW = None
        self.fc4.db = None
        self.fc4.dX = None
        self.fc4.dW = None
        self.act1.td = None
        self.act2.dt = None
        self.act3.dt = None
        self.act4.dt = None
        self.loss.dl = None
        

In [360]:
# гиперпараметры - скорость обучение, кол-во эпох обучения и размер батчей для обучения
lr = 0.01
epchos = 200
batch_size = 100

#инициализация модели
model = Model(X_train.shape[1], lr)


# нормаировка тренировочных и валидационных данных
mean = np.mean(X_train,axis=0)
std = np.std(X_train,axis=0)
X_train = (X_train[:,]-mean)/std

X_valid = (X_valid[:,]-mean)/std


# цикл обучения
for i in range(epchos):
    loss_epoch = []
    for j in range(len(X_train)//batch_size):
        X_batch = X_train[j*batch_size:j*batch_size + batch_size]
        y_batch = y_train[j*batch_size:j*batch_size + batch_size]
        
        preds = model.forward(X_batch)
        
        loss_batch = model.loss(preds,y_batch)
        loss_epoch.append(loss_batch)
        
        model.backward(y_batch)
        model.step()
    
    loss_epoch = np.mean(loss_epoch)
    
    #подсчет точности на валидационной выборке
    preds_valid = model.forward(X_valid)
    right = 0
    for k in range(len(X_valid)):
        if y_valid[k] - np.round(preds_valid[k]) < 0.001:
            right += 1
            
    accuracy = accuracy = right/len(y_valid)
    print('epoch :', i)
    print('loss :', loss_epoch)
    print('accuracy :', accuracy)
    print('------------------------------')

epoch : 0
loss : 141.45599033652715
accuracy : 0.7419354838709677
------------------------------
epoch : 1
loss : 51.83343408283896
accuracy : 0.7419354838709677
------------------------------
epoch : 2
loss : 46.27687447798594
accuracy : 0.7419354838709677
------------------------------
epoch : 3
loss : 44.31153832465681
accuracy : 0.7647058823529411
------------------------------
epoch : 4
loss : 43.20505100664984
accuracy : 0.7703984819734345
------------------------------
epoch : 5
loss : 42.774680683626215
accuracy : 0.7741935483870968
------------------------------
epoch : 6
loss : 42.43724758227205
accuracy : 0.7770398481973435
------------------------------
epoch : 7
loss : 42.13266343123093
accuracy : 0.7808349146110057
------------------------------
epoch : 8
loss : 41.84731026937053
accuracy : 0.7836812144212524
------------------------------
epoch : 9
loss : 41.54493799278756
accuracy : 0.7893738140417458
------------------------------
epoch : 10
loss : 41.15757477192785
ac

epoch : 85
loss : 21.132849593586126
accuracy : 0.8453510436432637
------------------------------
epoch : 86
loss : 20.10634811422181
accuracy : 0.9060721062618596
------------------------------
epoch : 87
loss : 20.349217550603722
accuracy : 0.8178368121442126
------------------------------
epoch : 88
loss : 19.12751928200168
accuracy : 0.8994307400379506
------------------------------
epoch : 89
loss : 22.444226452590833
accuracy : 0.8908918406072106
------------------------------
epoch : 90
loss : 19.51553218599623
accuracy : 0.8548387096774194
------------------------------
epoch : 91
loss : 18.91337594262847
accuracy : 0.8776091081593927
------------------------------
epoch : 92
loss : 18.00579735481319
accuracy : 0.818785578747628
------------------------------
epoch : 93
loss : 21.601584902724497
accuracy : 0.888045540796964
------------------------------
epoch : 94
loss : 18.25770967920233
accuracy : 0.8823529411764706
------------------------------
epoch : 95
loss : 17.2322670

epoch : 169
loss : 9.133857564346547
accuracy : 0.8690702087286527
------------------------------
epoch : 170
loss : 9.084524457252765
accuracy : 0.8690702087286527
------------------------------
epoch : 171
loss : 9.051344843541777
accuracy : 0.8681214421252372
------------------------------
epoch : 172
loss : 9.050747069084775
accuracy : 0.8690702087286527
------------------------------
epoch : 173
loss : 9.089069200885133
accuracy : 0.8709677419354839
------------------------------
epoch : 174
loss : 9.123747453688011
accuracy : 0.8709677419354839
------------------------------
epoch : 175
loss : 9.143765475717302
accuracy : 0.8709677419354839
------------------------------
epoch : 176
loss : 9.241032562131274
accuracy : 0.8738140417457305
------------------------------
epoch : 177
loss : 9.130097822546677
accuracy : 0.8719165085388995
------------------------------
epoch : 178
loss : 9.064551245691856
accuracy : 0.8719165085388995
------------------------------
epoch : 179
loss : 9